In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset=pd.get_dummies(dataset,dtype=int, drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
dataset=dataset.drop("User ID",axis=1)

In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [10]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_= sc.fit_transform(X_train)
X_test_= sc.transform(X_test)

In [12]:
from sklearn.naive_bayes import BernoulliNB

In [13]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'alpha': [0.1, 0.5, 1.0, 5.0, 10.0],
    'binarize': [0.0, 0.5, 1.0, 1.5, 2.0]
}

grid = GridSearchCV(BernoulliNB(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1') 
   
# fitting the model for grid search 
grid.fit(X_train_, y_train) 
 

Fitting 5 folds for each of 25 candidates, totalling 125 fits


GridSearchCV(estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 5.0, 10.0],
                         'binarize': [0.0, 0.5, 1.0, 1.5, 2.0]},
             scoring='f1', verbose=3)

In [14]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test_) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)


In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'alpha': 0.1, 'binarize': 0.5}: 0.8678975662538858


In [16]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[70 15]
 [ 3 46]]


In [17]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.96      0.82      0.89        85
           1       0.75      0.94      0.84        49

    accuracy                           0.87       134
   macro avg       0.86      0.88      0.86       134
weighted avg       0.88      0.87      0.87       134



In [18]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.49039615846338536

In [19]:
table=pd.DataFrame.from_dict(re)

In [20]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_binarize,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.025322,0.000906,0.005714,0.000513,0.1,0.0,"{'alpha': 0.1, 'binarize': 0.0}",0.588235,0.533333,0.303030,0.628571,0.562500,0.523134,0.114416,16
1,0.023875,0.001792,0.005150,0.000370,0.1,0.5,"{'alpha': 0.1, 'binarize': 0.5}",0.800000,0.837209,0.750000,0.904762,0.918919,0.842178,0.063414,1
2,0.011542,0.010127,0.005053,0.000590,0.1,1.0,"{'alpha': 0.1, 'binarize': 1.0}",0.709677,0.764706,0.666667,0.900000,0.888889,0.785988,0.093916,6
3,0.003312,0.000253,0.004811,0.000510,0.1,1.5,"{'alpha': 0.1, 'binarize': 1.5}",0.592593,0.461538,0.444444,0.848485,0.689655,0.607343,0.150291,11
4,0.003510,0.000557,0.004511,0.000640,0.1,2.0,"{'alpha': 0.1, 'binarize': 2.0}",0.480000,0.260870,0.100000,0.272727,0.347826,0.292285,0.123826,21
5,0.003639,0.000793,0.004560,0.000417,0.5,0.0,"{'alpha': 0.5, 'binarize': 0.0}",0.588235,0.533333,0.303030,0.628571,0.562500,0.523134,0.114416,16
6,0.003105,0.000660,0.005014,0.000707,0.5,0.5,"{'alpha': 0.5, 'binarize': 0.5}",0.800000,0.837209,0.750000,0.904762,0.918919,0.842178,0.063414,1
7,0.003208,0.000504,0.004614,0.000375,0.5,1.0,"{'alpha': 0.5, 'binarize': 1.0}",0.709677,0.764706,0.666667,0.900000,0.888889,0.785988,0.093916,6
8,0.003108,0.000209,0.005410,0.000856,0.5,1.5,"{'alpha': 0.5, 'binarize': 1.5}",0.592593,0.461538,0.444444,0.848485,0.689655,0.607343,0.150291,11
9,0.002265,0.000219,0.004455,0.000808,0.5,2.0,"{'alpha': 0.5, 'binarize': 2.0}",0.480000,0.260870,0.100000,0.272727,0.347826,0.292285,0.123826,21


In [23]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Gender Male 0 or 1:"))

Age: 28
Salary: 20000
Gender Male 0 or 1: 1


In [24]:
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
